In [ ]:
import sys, torch, os, time, imp
#sys.path.insert(0, "/home/marius/dm11/spikes/kilosort4/")
sys.path.insert(0, "/home/marius/dm11/spikes/kilosort_dev/")
import numpy as np
from kilosort import run_kilosort, io
import matplotlib.pyplot as plt

dev = torch.device('cuda:0')


In [ ]:
ops = {}
ops['root_config']   = '/media/marius/ssd1/github/Kilosort/configFiles'

ops['data_folder'] = '/media/marius/ssd1/data/spikes/sim_high_drift'
ops['config_fname']  = 'neuropixPhase3B1_kilosortChanMap.mat'

#ops['root_bin'] = '/media/marius/ssd1/data/spikes/Hopkins'
#ops['config_fname']  = 'neuropixPhase3A_kilosortChanMap.mat'

#ops['root_bin'] = '/media/marius/ssd1/data/spikes/ZAD29'
#ops['config_fname']  = 'neuropixPhase3B1_kilosortChanMap.mat'

ops['NchanTOT'] = 385
ops['fs']       = 30000

ops['nt']     =  61
ops['Th']       = 8
ops['spkTh']    = 8
ops['Th_detect']    = 9

ops['nwaves']   = 6
ops['nskip']    = 25
ops['nt0min']   = int(20 * ops['nt']/61)
ops['NT']       = 2 * ops['fs']
ops['nblocks']  = 5
ops['binning_depth'] = 5
ops['sig_interp'] = 20

probe  = io.load_probe(os.path.join(ops['root_config'], ops['config_fname']))
settings = ops 

t0= time.time()

In [ ]:
ops, st, tF, clu, Wall, is_ref = run_kilosort.run_kilosort(settings, probe, ops['data_folder'])

In [ ]:
from kilosort import bench

gt_path = os.path.join(ops['data_folder'] , "sim.imec0.ap_params.npz")
st_gt, clu_gt, yclu_gt, mu_gt, Wsub, nsp = bench.load_GT(ops, gt_path, nmax = 600)

st_new, clu_new, yclu_new, Wsub_new = bench.convert_ks_output(ops, st, clu)

fmax, fmiss, fpos = bench.compare_recordings(st_gt, clu_gt, yclu_gt, st_new, clu_new, yclu_new)

In [ ]:
fpath ='/home/marius/dm11_stringer/spiking/sim_high_drift/outputs/ironclust'
st_new, clu_new, yclu_new, Wsub = bench.load_phy(fpath, ops)

fmax, fmiss, fpos = bench.compare_recordings(st_gt, clu_gt, yclu_gt, st_new, clu_new, yclu_new)



In [ ]:
kk = 0
#kk += 1
print(kk, (clu==kk).sum())

import matplotlib.pyplot as plt 

wav = ops['wPCA'].cpu().numpy().T @ Wsub_new[kk]
print((wav**2).sum()**.5)
plt.plot(wav);

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(2,4,1)
plt.plot(np.sort(fmax)[::-1])
print((fmax>.8).sum())

plt.subplot(2,4,2)
plt.semilogx(nsp, fmax, 'o')

plt.subplot(2,4,3)
plt.semilogx(mu_gt, fmiss, 'o')

plt.subplot(2,4,4)
plt.semilogx(mu_gt, fpos, 'o')

plt.subplot(2,4,5)
plt.semilogx(mu_gt, fmax, 'o')

In [ ]:
unq_clu, nsp = np.unique(clu_gt, return_counts = True)
np.abs(np.diff(unq_clu) - 1).sum()